In [76]:
# Module Imports
import pandas as pd
import sqlite3
import os
from os import path
from datetime import date
from IPython.core.interactiveshell import InteractiveShell

In [77]:
# Display all columns of DataFrames
pd.options.display.max_columns = None

# Print all output in a cell not just the last piece of output
InteractiveShell.ast_node_interactivity = "all"

# Data directory for csv files
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

# Dictionary relating prop names to stats
stat_dict = {'Points': 'PTS', 'Assists': 'AST', 'Rebounds': 'TRB', 'Reb + Ast': 'TRB + AST', 'Made Threes': '3P', 'Pts + Ast': 'PTS + AST', 'Pts + Reb': 'PTS + TRB', 'Pts + Reb + Ast': 'PTS + TRB + AST'}

# Open database connection
conn = sqlite3.connect(path.join(DATA_DIR, 'nba.sqlite'))

In [78]:
# Get today's date in our format and print it out
today = date.today()
datef = today.strftime("%Y%m%d")
print(datef)

# Load prop data for today into a DataFrame
try:
    props_df = pd.read_csv(path.join(DATA_DIR, 'player_props', f'{datef} - new.csv'))
except:
    os.system('python new_get_player_props.py')
    props_df = pd.read_csv(path.join(DATA_DIR, f'{datef} - new.csv'))

# Clean prop table by dropping old index column and converting columns to numeric
props_df = props_df.drop(props_df.columns[0], axis = 1)
for col in props_df.columns:
    try:
        props_df[col] = pd.to_numeric(props_df[col])
    except:
        continue
props_df

20210720


,Player,Prop,Over,Over Odds,Under,Under Odds
0,bookede01,Points,31.5,-110,31.5,-110
1,paulch01,Points,21.5,-104,21.5,-116
2,aytonde01,Points,16.5,-110,16.5,-110
3,bridgmi01,Points,11.5,104,11.5,-126
4,crowdja01,Points,10.5,-118,10.5,-104
...,...,...,...,...,...,...
80,antetgi01,Pts + Reb + Ast,51.5,-122,51.5,-104
81,middlkh01,Pts + Reb + Ast,38.5,-106,38.5,-120
82,holidjr01,Pts + Reb + Ast,33.5,-108,33.5,-118
83,lopezbr01,Pts + Reb + Ast,16.5,-113,16.5,-113


In [79]:
# Loop through each row of the props DataFrame
for ind in props_df.index:
    # Get the prop name to use in the query as well as the player name
    prop_parts = stat_dict[props_df.loc[ind, 'Prop']].split(' + ')
    prop_name = f'''"{'" + "'.join(prop_parts)}"'''
    player = props_df.loc[ind, 'Player']
    
    # Get the player's table for that prop
    table = pd.read_sql(f'''SELECT {prop_name} FROM player_games JOIN games ON player_games.game_id = games.game_id WHERE player_id = "{player}" AND Season = "2020-21" AND Playoffs = 1''', conn)
    for col in table.columns:
        try:
            table[col] = pd.to_numeric(table[col], errors='coerce')
        except:
            continue
    
    # Reset the prop name for single props
    if len(prop_parts) == 1:
        prop_name = prop_name[1:len(prop_name) - 1]
    
    # Create average and cover columns
    props_df.loc[ind, 'Playoff Avg'] = table[prop_name].mean()
    props_df.loc[ind, 'Finals Avg'] = table[-5:][prop_name].mean()
    props_df.loc[ind, 'Playoff Cover %'] = (table[prop_name] > props_df.loc[ind, 'Over']).mean()
    props_df.loc[ind, 'Finals Cover %'] = (table.iloc[-5:][prop_name] > props_df.loc[ind, 'Over']).mean()

In [80]:
# Print whole props table and highlight over/under cover percentages
props_df.style.apply(lambda x: [ "background: green" if v >= 0.5 else "background: red" for v in x ], axis = 1, subset = ['Playoff Cover %', 'Finals Cover %'])

,Player,Prop,Over,Over Odds,Under,Under Odds,Playoff Avg,Finals Avg,Playoff Cover %,Finals Cover %
0,bookede01,Points,31.500000,-110,31.500000,-110,27.714286,30.000000,0.285714,0.400000
1,paulch01,Points,21.500000,-104,21.500000,-116,18.842105,21.000000,0.315789,0.400000
2,aytonde01,Points,16.500000,-110,16.500000,-110,15.952381,15.200000,0.523810,0.600000
3,bridgmi01,Points,11.500000,104,11.500000,-126,11.285714,13.000000,0.428571,0.600000
4,crowdja01,Points,10.500000,-118,10.500000,-104,10.571429,11.000000,0.476190,0.600000
5,antetgi01,Points,32.500000,-116,32.500000,-104,29.200000,32.200000,0.450000,0.400000
6,middlkh01,Points,25.500000,-110,25.500000,-110,23.863636,25.400000,0.409091,0.600000
7,holidjr01,Points,19.500000,-118,19.500000,-104,17.590909,17.600000,0.363636,0.400000
8,lopezbr01,Points,11.500000,100,11.500000,-122,13.090909,11.800000,0.500000,0.400000
9,tuckepj01,Points,4.500000,-104,4.500000,-118,4.454545,4.800000,0.454545,0.600000


In [81]:
# Print the extreme cover values
props_df[(props_df['Playoff Cover %'] > 0.5) & (props_df['Finals Cover %'] > 0.5)].sort_values(by = ['Playoff Cover %'], ascending = False)
props_df[(props_df['Finals Cover %'] > 0.5)]

,Player,Prop,Over,Over Odds,Under,Under Odds,Playoff Avg,Finals Avg,Playoff Cover %,Finals Cover %
21,lopezbr01,Rebounds,4.5,-132,4.5,108,5.772727,4.8,0.772727,0.6
72,holidjr01,Reb + Ast,13.5,-102,13.5,-125,14.136364,14.6,0.681818,0.8
12,payneca01,Points,6.5,102,6.5,-126,9.285714,6.8,0.666667,0.6
32,holidjr01,Assists,7.5,-144,7.5,118,8.545455,9.0,0.636364,0.6
39,middlkh01,Made Threes,2.5,-150,2.5,118,2.681818,3.0,0.636364,0.8
42,tuckepj01,Made Threes,0.5,-265,0.5,200,0.863636,0.8,0.636364,0.8
73,lopezbr01,Reb + Ast,5.5,110,5.5,-140,6.045455,5.0,0.636364,0.6
41,lopezbr01,Made Threes,0.5,-245,0.5,186,1.045455,1.0,0.590909,0.6
29,crowdja01,Assists,1.5,-124,1.5,102,1.952381,2.0,0.571429,0.6
36,bridgmi01,Made Threes,1.5,-162,1.5,126,1.666667,1.8,0.571429,0.6


,Player,Prop,Over,Over Odds,Under,Under Odds,Playoff Avg,Finals Avg,Playoff Cover %,Finals Cover %
2,aytonde01,Points,16.5,-110,16.5,-110,15.952381,15.2,0.523810,0.6
3,bridgmi01,Points,11.5,104,11.5,-126,11.285714,13.0,0.428571,0.6
4,crowdja01,Points,10.5,-118,10.5,-104,10.571429,11.0,0.476190,0.6
6,middlkh01,Points,25.5,-110,25.5,-110,23.863636,25.4,0.409091,0.6
9,tuckepj01,Points,4.5,-104,4.5,-118,4.454545,4.8,0.454545,0.6
10,connapa01,Points,8.5,-108,8.5,-114,7.181818,11.0,0.454545,0.6
12,payneca01,Points,6.5,102,6.5,-126,9.285714,6.8,0.666667,0.6
17,crowdja01,Rebounds,6.5,-114,6.5,-106,5.761905,7.6,0.333333,0.6
21,lopezbr01,Rebounds,4.5,-132,4.5,108,5.772727,4.8,0.772727,0.6
23,connapa01,Rebounds,4.5,-142,4.5,116,4.227273,5.4,0.363636,0.6
